In [1]:
# adjust the R limit
memory.limit(size= 81011630)

[1] 81011630

In [1]:
#necessary library
library(data.table) #fread
library(lubridate)
library(reshape2)
library(dplyr)
library(scales)
library(stringr)
#library(tidyverse)

Warning message:
"package 'data.table' was built under R version 3.6.3"

Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following object is masked from 'package:base':

    date


Warning message:
"package 'reshape2' was built under R version 3.6.3"

Attaching package: 'reshape2'


The following objects are masked from 'package:data.table':

    dcast, melt



Attaching package: 'dplyr'


The following objects are masked from 'package:lubridate':

    intersect, setdiff, union


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [9]:
# Select the month you want to investigate
YEARLIST =("2019")
MONTHLIST = c("04")
COMPUTATION_FILEPATH = "../../data/tidy/"

## Bin speeds and aggregate table

In [2]:
interval_df = read.csv("../../data/tidy/trajectory-19-04-.csv")
dc = fread("../../data/raw/energy-consumption-08-19.csv") # Read in energy data
d_ridership = read.csv("../../data/raw/ridership-2019-2020-.csv")# Read in ridership data

In [ ]:
# aggregrate_trajectory_table
line_computation = function(year, month){
    assign( fread(paste(DISTANCE_FILEPATH, paste( "trajectories", month, year, ".csv", sep = "-", collapse = ""), sep="")))
    assign("dh", fread(paste(DISTANCE_FILEPATH, paste("heavyrail", "trajectories", month, year, ".csv", sep = "-", collapse = ""), sep="")))
    # Combine the original tables to a single one for analysis
    dg = subset(dg, select = c(trxtime, trainid, lineid, lat, lon))
    dh = subset(dh, select = c(trxtime, trainid, lineid, lat, lon))
    df = rbind(dg, dh)
    return(df)
}

In [22]:
# select random days
set.seed(1234)
sub = sample(1:nrow(interval_df),720)
interval_df_test = interval_df[sub,]

In [3]:
# Unit conversion
unit_transfer = function(df){
   df$hour = hour(df$trxtime)
   df$speed_mph = df$speed_kph*0.621371 #kph to mph
   df$distance_mile = df$dist_meters*0.000621371 #convert from meters to mile
   df$time_hr = df$interval_seconds/3600.0 #convert from seconds to hour
  return(df)
}    

In [4]:
interval_df = unit_transfer(interval_df)

In [5]:
head(interval_df)

,X,trxtime,trainid,lineid,lat,lon,day,month,year,dist_meters,interval_seconds,speed_mps,speed_kph,accel_mps2,hour,speed_mph,distance_mile,time_hr
,<int>,<fct>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,1,2019-04-01 04:02:25,10001,4,42.33701,-71.25349,1,4,2019,NA,NA,NA,NA,NA,4,NA,NA,NA
2,2,2019-04-01 04:02:37,10001,4,42.33710,-71.25371,1,4,2019,20.55303,12,1.71275229,6.16590825,NA,4,3.83131658,0.01277106,0.003333333
3,3,2019-04-01 04:11:44,10001,4,42.33701,-71.25349,1,4,2019,20.55303,547,0.03757409,0.13526673,-3.062483e-03,4,0.08405082,0.01277106,0.151944444
4,4,2019-04-01 04:27:21,10001,4,42.33710,-71.25371,1,4,2019,20.55303,937,0.02193493,0.07896574,-1.669068e-05,4,0.04906702,0.01277106,0.260277778
5,5,2019-04-01 04:28:46,10001,4,42.33701,-71.25349,1,4,2019,20.55303,85,0.24180032,0.87048117,2.586652e-03,4,0.54089175,0.01277106,0.023611111
6,6,2019-04-01 04:44:12,10001,4,42.33710,-71.25371,1,4,2019,20.55303,926,0.02219549,0.07990378,-2.371542e-04,4,0.04964989,0.01277106,0.257222222


In [10]:
interval_df = bin_speeds(interval_df,5)

[1] "The speed bins are: 0"               
[2] "The speed bins are: 6.56858695628579"
[3] "The speed bins are: 12.7370220967312"
[4] "The speed bins are: 21.8533150410368"
[5] "The speed bins are: 29.0026902865978"
[6] "The speed bins are: 79.5312028066899"
[1] "Percentage error of summed speed bin times = 0%"


In [11]:
interval_df = bin_accelerations(interval_df,5)

[1] "The acceleration bins are: -5.99773290866069"  
[2] "The acceleration bins are: -0.247784796968302" 
[3] "The acceleration bins are: -0.0443615261657048"
[4] "The acceleration bins are: 0.0598579974871787" 
[5] "The acceleration bins are: 0.610563396512518"  
[6] "The acceleration bins are: 5.99942097861047"   
[1] "Percentage error of summed acceleration bin times = -4.31%"


In [7]:
# Calculate the speed bins 
bin_speeds <- function (dataframe, num_bins) {
  dataframe = data.table(dataframe)
  dummy_cols = c(paste0("speed_bin_",1:num_bins,"_dummy"))
  bin_time_cols = c(paste0("speed_bin_",1:num_bins,"_time_hr"))
  cutpoints <- quantile(dataframe$speed_mph,seq(0, 1, 1/num_bins),na.rm=TRUE) 
  print(paste0("The speed bins are: ", cutpoints))
  for(n in seq(1, num_bins)) {
    if(n == 1){
     dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x < cutpoints[n+1], 1, 0))]
    }
    else if (n == num_bins){
     dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x >= cutpoints[num_bins], 1, 0))]
    }
    else {
    dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x >= cutpoints[n] & x < cutpoints[n + 1], 1, 0))]
    }
  }
 dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
 print(paste0("Percentage error of summed speed bin times = ", 
              round(100*(sum(colSums(dataframe %>% select(starts_with("speed_bin_") & ends_with("_time_hr")),na.rm=TRUE)) 
                         - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))
 return(dataframe) 
}

In [8]:
# Calculate the acceleration bins 
bin_accelerations <- function (dataframe, num_bins) {
  dataframe = data.table(dataframe)
  dummy_cols = c(paste0("accel_bin_",1:num_bins,"_dummy"))
  bin_time_cols = c(paste0("accel_bin_",1:num_bins,"_time_hr"))
  cutpoints <- quantile(dataframe$accel_mps2,seq(0, 1, 1/num_bins),na.rm=TRUE) 
  print(paste0("The acceleration bins are: ", cutpoints))
  for(n in seq(1, num_bins)) {
    if(n == 1){
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x < cutpoints[n+1], 1, 0))]
    }
    else if (n == num_bins){
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x >= cutpoints[num_bins], 1, 0))]
    }
    else {
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x >= cutpoints[n] & x < cutpoints[n + 1], 1, 0))]
    }
  }
  dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
  print(paste0("Percentage error of summed acceleration bin times = ", 
               round(100*(sum(colSums(dataframe %>% select(starts_with("accel_bin_") & ends_with("_time_hr")),na.rm=TRUE)) 
                          - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
  return(dataframe)
}

In [13]:
# Aggregate dataframe at hour level
hour_aggregate <- function (dataframe,num_bins) {
    dataframe = data.table(dataframe)
    dataframe$month = as.character(dataframe$month)
    dataframe$hour = as.character(dataframe$hour)
    dataframe$day = as.character(dataframe$day)
    sum_cols = c("distance_mile","time_hr",paste0("speed_bin_",1:num_bins,"_time_hr"),paste0("accel_bin_",1:num_bins,"_time_hr"))
    agg_df = dataframe[, lapply( .SD, sum , na.rm=TRUE), by = c("month",'hour',"day"), .SDcols = sum_cols]
    avg_interval_speed_mph_df = dataframe[, lapply( .SD, mean , na.rm=TRUE), by = c("month","hour","day"), .SDcols = 'speed_mph']
    agg_df[, 'avg_interval_speed_mph'] = avg_interval_speed_mph_df$speed_mph
    agg_df[, 'avg_hour_speed_mph'] = agg_df$distance_mile/agg_df$time_hr
    return(agg_df)
}

In [14]:
interval_df = hour_aggregate(interval_df,5)

In [45]:
head(interval_df)

month,hour,day,distance_mile,time_hr,speed_bin_1_time_hr,speed_bin_2_time_hr,speed_bin_3_time_hr,speed_bin_4_time_hr,speed_bin_5_time_hr,accel_bin_1_time_hr,accel_bin_2_time_hr,accel_bin_3_time_hr,accel_bin_4_time_hr,accel_bin_5_time_hr,avg_interval_speed_mph,avg_hour_speed_mph
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,4,1,50.87189,7.628333,5.708889,0.5730556,0.4316667,0.4094444,0.5052778,0.2994444,1.061111,4.610833,0.5627778,0.142500,19.98537,6.668808
4,6,1,1105.22442,106.597778,56.297500,17.1316667,14.3844444,9.2702778,9.5113889,9.6311111,28.485278,41.570833,14.8036111,4.720556,19.69621,10.368175
4,7,1,1347.50198,134.801667,72.450278,23.2561111,16.9636111,10.9122222,11.2194444,12.2600000,40.182778,53.694167,18.5361111,6.105556,19.37512,9.996182
4,5,1,484.39167,63.953611,41.391944,8.7316667,6.0686111,3.6436111,4.1083333,4.2600000,11.321389,25.282222,6.6133333,1.966389,18.83070,7.574110
4,8,1,1159.65304,133.512222,78.943889,21.0233333,15.5844444,9.3744444,8.5861111,9.7563889,37.225833,64.064722,15.7836111,5.729722,18.37129,8.685744
4,9,1,1287.95919,169.110833,107.528056,25.1430556,17.6358333,9.5366667,9.2672222,11.1069444,39.970000,87.192778,17.9300000,5.674722,17.97371,7.616066


In [46]:
# Combine ridership data
merge_ridership = function(merged_dt,d_ridership){
   d_ridership$route_or_line = as.character(d_ridership$route_or_line)
   d_ridership$route_or_line[which(d_ridership$route_or_line == "Green Line")] = "4"
   d_ridership$route_or_line[which(d_ridership$route_or_line == "Red Line")] = "1"
   d_ridership$route_or_line[which(d_ridership$route_or_line == "Blue Line")] = "2"
   d_ridership$route_or_line[which(d_ridership$route_or_line == "Orange Line")] = "3"
   d_ridership$year = as.character(year(d_ridership$servicedate)）
   d_ridership$month = as.character(month(d_ridership$servicedate)）
   d_ridership$day = as.character(day(d_ridership$servicedate)）
   d_ridership$hour = as.character(hour(d_ridership$halfhour)）
   d_ridership = aggregate(d_ridership$rawtaps_split, list(year = d_ridership$year, month = d_ridership$month, day = d_ridership$day, hour = d_ridership$hour), FUN = sum)
   names(d_ridership)[names(d_ridership) == 'x'] <- 'ridership'
   #d_ridership$lineid = as.integer(d_ridership$lineid) 
   d_ridership = subset(d_ridership,year == "2019" & month == "4")
   merged_db = merge(merged_dt,d_ridership,by = c("month","day","hour"),all=T) 
   return(merged_db)
}

In [47]:
df_combin = merge_ridership(interval_df,d_ridership)

In [74]:
# Combine energy consumption data
merge_energy <- function (energy_df,hour_dt) {
    energy_df = subset(energy_df,energy_df$Year == 2019 & energy_df$Month == 4)
    # Melt by hour 
    melted_energy_df = melt(energy_df, id.vars=c('Year','Month','Day of Month','WJ','TAVG'), measure.vars = paste0("Hour ",1:24))
        colnames(melted_energy_df) = c('year', 'month', 'day', 'weekends_weekday', 'TAVG', 'Hour', 'energy_kwh')
    hour_energy_dt <- setDT(melted_energy_df)
    hour_energy_dt[, Hour := str_replace(Hour, "Hour ", "")]
    hour_energy_dt$Hour = as.numeric(hour_energy_dt$Hour)
    hour_energy_dt$Hour = hour_energy_dt$Hour - 1
    colnames(hour_energy_dt)[6] <- 'hour'
    hour_energy_dt$year = as.character(hour_energy_dt$year)
    hour_energy_dt$month = as.character(hour_energy_dt$month)
    hour_energy_dt$day = as.character(hour_energy_dt$day)
    hour_energy_dt$hour = as.character(hour_energy_dt$hour)
    merged_dt = merge(hour_dt,hour_energy_dt,by = c("year","month","day","hour") , all = T)
    return(merged_dt)
}

In [75]:
df_main = merge_energy(dc,df_combin)

In [76]:
head(df_main)

year,month,day,hour,distance_mile,time_hr,speed_bin_1_time_hr,speed_bin_2_time_hr,speed_bin_3_time_hr,speed_bin_4_time_hr,...,accel_bin_2_time_hr,accel_bin_3_time_hr,accel_bin_4_time_hr,accel_bin_5_time_hr,avg_interval_speed_mph,avg_hour_speed_mph,ridership,weekends_weekday,TAVG,energy_kwh
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
NA,5,1,0,766.80982,20518.943889,20488.48333,10.211944,8.805278,5.849722,...,14.068333333,19059.505833,8.20055556,3.3375000,21.70410,0.03737082,NA,NA,NA,NA
NA,5,1,1,224.00092,1433.453056,1424.07694,2.443333,3.037500,1.813333,...,2.736944444,1424.177222,1.35333333,1.2955556,26.08955,0.15626666,NA,NA,NA,NA
NA,5,1,2,83.01897,5.771944,1.76750,1.313056,1.537222,0.550000,...,0.001666667,3.003056,0.01305556,0.5802778,23.04987,14.38318945,NA,NA,NA,NA
2019,4,1,0,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,1561.0,2,42,40.567
2019,4,1,1,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,171.7,2,42,35.110
2019,4,1,10,1157.92787,117.502778,63.17194,21.106944,15.074167,8.828889,...,33.216944444,52.217500,15.46083333,5.1025000,19.38688,9.85447235,19134.9,2,42,53.894


In [77]:
write.csv(df_main,"../../data/tidy/system-model-19-04.csv")

## Computing the final table

In [59]:
main <- function (interval_df,num_bins,energy_df,d_ridership,yy,mm) {
    final_df <- interval_df %>% unit_transfer（) %>% bin_speeds(num_bins) %>% 
    bin_accelerations(num_bins) %>% merge_ridership(d_ridership,yy,mm) %>% merge_energy(energy_df,yy,mm)
    return(final_df)
}

In [ ]:
df_final = main(interval_df,5,energy_df,d_ridership)
#write.csv(df_final,"../Data/tidy/system-model.csv")

In [106]:
head(dm)

,year,month,day,hour,lineid,ridership,distance_mile,time_hr,speed_bin_1_time_hr,speed_bin_2_time_hr,...,accel_bin_1_time_hr,accel_bin_2_time_hr,accel_bin_3_time_hr,accel_bin_4_time_hr,accel_bin_5_time_hr,avg_interval_speed_mph,avg_hour_speed_mph,weekends_weekday,TAVG,energy_kwh
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2019,4,1,0,4,448.77,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2,42,40.567
2,2019,4,1,0,1,477.41,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2,42,40.567
3,2019,4,1,0,2,184.70,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2,42,40.567
4,2019,4,1,0,3,450.12,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2,42,40.567
5,2019,4,1,1,2,21.40,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2,42,35.110
6,2019,4,1,1,3,61.35,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2,42,35.110


# Experimental section

In [ ]:
# # Calculate the acceleration bins
# bin_accelerations <- function (dataframe) {
#     dummy_cols = c(paste0("accel_bin_",1:5,"_dummy"))
#     bin_time_cols = c(paste0("speed_bin_",1:5,"_time_hr"))
#     cutpoints  <- quantile(dataframe$accel_mps2,seq(0,1,.2),na.rm=TRUE) 
#     print(paste0("The acceleration bins are: ", testcuts))
#     dataframe[, 'accel_bin_1_dummy' := sapply(accel_mps2,  function(x) ifelse (x < testcuts[2], 1, 0))]
#     dataframe[, 'accel_bin_2_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[2] & x < testcuts[3], 1, 0))]
#     dataframe[, 'accel_bin_3_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[3] & x < testcuts[4], 1, 0))]
#     dataframe[, 'accel_bin_4_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[4] & x < testcuts[5], 1, 0))]
#     dataframe[, 'accel_bin_5_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[5], 1, 0))]
#     dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
#     print(paste0("Percentage error of summed acceleration bin times = ", round(100*(sum(colSums(dataframe[, bin_time_cols ],na.rm=TRUE)) 
#                - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
#     return(dataframe)
# }